In [1]:
from shogun import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [17]:
df = pd.read_csv('../data/nyc-sales.csv')
df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [3]:
for i in range(len(df)):
    if(df.loc[i,'SALE PRICE'] == ' -  '):
        df.at[i,'SALE PRICE'] = np.nan
        
# test is the data with 'SALE PRICE' == NaN
df_test = df[df['SALE PRICE'].isna()]

#train is the cleaned data
df_train = df[df['SALE PRICE'].notna()]


In [4]:
# training set
col_x = [col for col in df.columns
      if col != 'SALE PRICE']
X_train = df_train[col_x]

In [5]:
# target variable
target = ['SALE PRICE']
y_train = df_train[target]


In [7]:
# imputer not required
# myimputer = SimpleImputer(strategy='median')  

encoder = LabelEncoder()

s = (X_train.dtypes == 'object')

cat_vars = list(s[s].index)

# print(cat_vars)

In [8]:
# drop the unwanted columns
X_new = X_train.drop(columns=['Unnamed: 0','NEIGHBORHOOD','ADDRESS','EASE-MENT','BUILDING CLASS CATEGORY','SALE DATE','APARTMENT NUMBER'])

# X_new.head()

In [9]:
s = (X_new.dtypes == 'object')
cat_varsx = list(s[s].index)
# print(cat_varsx)

In [18]:
#  Labeling properly the columns with object data type
le = LabelEncoder()

label_X = X_new.copy()
for col in cat_varsx:
    label_X[col] = le.fit_transform(X_train[col])

# label_X contains the cleaned up training set

,BOROUGH,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE
0,1,6,392,6,16,10009,5,0,5,916,4378,1900,2,15
3,1,7,402,21,18,10009,10,0,10,1712,4464,1913,2,17
4,1,6,404,55,16,10009,6,0,6,1828,3728,1900,2,15
6,1,7,406,32,18,10009,8,0,8,1070,3537,1920,2,17
9,1,5,387,153,34,10009,24,0,24,3615,1265,1920,2,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,5,1,7349,34,13,10309,2,0,2,1864,2119,1998,1,12
84544,5,1,7349,78,13,10309,2,0,2,1976,1904,1998,1,12
84545,5,1,7351,60,11,10309,2,0,2,3287,775,1925,1,10
84546,5,10,7100,28,82,10309,0,7,7,1482,4372,2001,4,82


In [19]:
#Dropping some data to account for Memory error, see issue #2
label_X = label_X[:5000]
y_train = y_train[:5000]


In [20]:
# contains the error

from time import time
from sklearn.kernel_ridge import KernelRidge

t0 = time()
skr = KernelRidge(kernel='rbf', gamma=0.1)
skr.fit(label_X, y_train)
tsk1 = time() - t0
print(tsk1)

2.641537666320801


In [13]:
#Convert features and labels suitable for shogun

label_X= label_X.to_numpy().astype(float)
y_train = y_train.to_numpy().astype(float)
y_train = y_train.reshape(y_train.shape[0],) #reshape to (num_samples,)

In [14]:
from shogun import *


train_features = RealFeatures(label_X.T)
labels_train = RegressionLabels(y_train)


width = 0.1
kernel = GaussianKernel(train_features, train_features, width)
tau = 1

t0=time()
krr = KernelRidgeRegression(tau, kernel, labels_train)
krr.train()
tsg1=time()-t0

print(tsg1) 

66.37942337989807


In [15]:
#Final remarks
print('----------------------------------------')
print("############ Final remarks  ############")
print('----------------------------------------')
print("\n")
print("Sklearn training time: " + str(tsk1))
print("Shogun training time: " + str(tsg1))
print("Performance gap: " + str(tsg1 - tsk1))
print("\n")
print('----------------------------------------')


----------------------------------------
############ Final remarks  ############
----------------------------------------


Sklearn training time: 20.580644369125366
Shogun training time: 66.37942337989807
Performance gap: 45.798779010772705


----------------------------------------
